# Test LD module

In [1]:
import Base.Threads.@threads

using Chain
using DLMReader
using Distributions
using GLM
using Glob
using InMemoryDatasets
using IterTools
using Random
using StatsBase
using SnpArrays

In [2]:
include("../src/ld.jl")

formatSnpData!

### 1) get all snps around il7

In [4]:
gwas_file = "/mnt/r6_data/GWAS_MRsuite/input_MRsuite_hypoT4_SM"
chr = 5
tss = 35852695

gwas_data = filereader(gwas_file, 
                       header = [:chr, :pos, :a_effect_out, :a_other_out, :β_out, :se_out, :pval_out],
                       types = Dict( 1=>Int8, 2=>Int32, 3=>String, 4=>String, 
                                     5=>Float64, 6=>Float64, 7=>Float64),
                       delimiter = '\t', skipto = 2)

,chr,pos,a_effect_out,a_other_out,β_out,se_out,pval_out
,identity,identity,identity,identity,identity,identity,identity
,Int8?,Int32?,String?,String?,Float64?,Float64?,Float64?
1,1,565987,T,C,-0.159782,0.089708,0.07489
2,1,566139,A,C,0.155926,0.132123,0.2379
3,1,568665,A,G,-0.054151,0.057159,0.3434
4,1,592368,A,G,-0.061272,0.063967,0.3381
5,1,662531,A,G,0.04487,0.054755,0.4125
6,1,662613,A,G,-0.03076,0.038043,0.4188
7,1,662622,A,G,-0.005365,0.012785,0.6748
8,1,666249,T,C,0.155111,0.124543,0.213


In [6]:
in_window(s::SubArray) = (s[1] == chr && abs(s[2]-tss)≤500000 && s[3]<5e-3)

filter!(gwas_data, [:chr, :pos, :pval_out], type = in_window)
sort!(gwas_data, :pval_out)

,chr,pos,a_effect_out,a_other_out,β_out,se_out,pval_out
,identity,identity,identity,identity,identity,identity,identity
,Int8?,Int32?,String?,String?,Float64?,Float64?,Float64?
1,5,35846815,T,G,-0.059365,0.008319,9.621e-13
2,5,35837234,A,G,-0.059342,0.00832,9.856e-13
3,5,35841449,A,G,0.059324,0.00832,1.005e-12
4,5,35844125,A,G,-0.059285,0.00832,1.036e-12
5,5,35843832,C,G,0.059281,0.00832,1.038e-12
6,5,35833268,T,C,0.055793,0.00806,4.454e-12
7,5,35803577,T,C,-0.056192,0.008344,1.642e-11
8,5,35833717,A,G,0.054111,0.008138,2.941e-11


### 2) get ld matrix

In [5]:
chr_pos = collect(zip(gwas_data.chr, gwas_data.pos))

40-element Vector{Tuple{Any, Any}}:
 (5, 35846815)
 (5, 35837234)
 (5, 35841449)
 (5, 35844125)
 (5, 35843832)
 (5, 35833268)
 (5, 35803577)
 (5, 35833717)
 (5, 35805208)
 (5, 35806110)
 (5, 35866218)
 (5, 35807455)
 (5, 35814105)
 ⋮
 (5, 35857850)
 (5, 35879095)
 (5, 35861152)
 (5, 35857704)
 (5, 35881376)
 (5, 35862841)
 (5, 35850149)
 (5, 35853319)
 (5, 35852311)
 (5, 35810561)
 (5, 35811282)
 (5, 35851831)

In [12]:
files = ["/home/samat33/CrickData_samat33/PLINK/EUR_biallelic/chr$(i)_CEU.BIALL" for i in 1:22]

GenotypesArr = [SnpData(SnpArrays.datadir(file)) for file in files]

@threads for i in 1:lastindex(GenotypesArr)
    GenotypesArr[i].snp_info.idx = collect(1:size(GenotypesArr[i].snp_info, 1))
    GenotypesArr[i].snp_info.chr_pos = collect(
        zip(parse.(Int8, GenotypesArr[i].snp_info.chromosome), 
        GenotypesArr[i].snp_info.position)
    )
    sort!(GenotypesArr[i].snp_info, :chr_pos)
end

In [ ]:
R2_mat, found_v_b = getLDmat(GenotypesArr[5], chr_pos)

In [ ]:
R2_mat

### 3) Clumping

In [7]:
kept_v_b = clump(GenotypesArr[5], chr_pos)

LoadError: MethodError: no method matching clump(::SnpData, ::Vector{Tuple{Any, Any}})
[0mClosest candidates are:
[0m  clump(::SnpData, [91m::AbstractArray{Tuple{T1, T2}, 1}[39m) where {T1, T2<:Integer} at ~/miniconda3/envs/julia_r/share/julia/dev/MrPainter/src/ld.jl:133
[0m  clump(::SnpData, [91m::AbstractArray{Tuple{T1, T2}, 1}[39m, [91m::Float64[39m) where {T1, T2<:Integer} at ~/miniconda3/envs/julia_r/share/julia/dev/MrPainter/src/ld.jl:133

## Form Interval pQTL

In [29]:
qtl_file = "/home/samat33/CrickData_samat33/Interval/meta_filtered_final/IL7R.5089.11.3/IL7R.5089.11.3_chrom_5_meta_final_v1.tsv"

chr = 5
tss = 35852695

qtl_data = filereader(qtl_file, 
                       header = [:id_, :chr, :pos, :a_effect_out, :a_other_out, :β_out, :se_out, :pval_out],
                       types = Dict( 2=>Int8, 3=>Int32, 4=>String, 5=>String,
                                     6=>Float64, 7=>Float64, 8=>Float64),
                       delimiter = '\t', skipto = 2)

,id_,chr,pos,a_effect_out,a_other_out,β_out,se_out,pval_out
,identity,identity,identity,identity,identity,identity,identity,identity
,String?,Int8?,Int32?,String?,String?,Float64?,Float64?,Float64?
1,5_10056_3,5,10056,a,c,-0.0005,0.0547,-0.0
2,5_10063_6,5,10063,a,c,0.0101,0.0376,-0.1
3,5_10099_10,5,10099,a,c,-0.0061,0.0385,-0.06
4,5_10116_13,5,10116,a,t,-0.0578,0.1479,-0.16
5,5_10219_14,5,10219,a,c,-0.0146,0.0471,-0.12
6,5_10231_15,5,10231,a,c,-0.0054,0.0737,-0.03
7,5_10237_17,5,10237,a,g,-0.083,0.0712,-0.61
8,5_10249_18,5,10249,a,c,0.0028,0.0404,-0.02


In [30]:

qtl_data = @chain qtl_data begin
    modify(:pval_out => x -> exp10.(x))
    filter([:chr, :pos, :pval_out], type = in_window)
    filter(:a_effect_out, type = x -> length(x) == 1)
    filter(:a_other_out, type = x -> length(x) == 1)
    sort(:pval_out)
end


chr_pos = collect(zip(qtl_data.chr, qtl_data.pos))

994-element Vector{Tuple{Any, Any}}:
 (5, 35866218)
 (5, 35883757)
 (5, 35883251)
 (5, 35883176)
 (5, 35883804)
 (5, 35812414)
 (5, 35827665)
 (5, 35814105)
 (5, 35811768)
 (5, 35827646)
 (5, 35811973)
 (5, 35810268)
 (5, 35807455)
 ⋮
 (5, 35850278)
 (5, 35876251)
 (5, 35474284)
 (5, 35497748)
 (5, 35478774)
 (5, 35696508)
 (5, 35712138)
 (5, 35529733)
 (5, 35480468)
 (5, 35656344)
 (5, 35712310)
 (5, 35713629)

In [13]:
R2_mat, found_v_b = getLDmat(GenotypesArr[5], chr_pos)

([1.0 0.9563998283455755 … 0.21129976679088458 0.21088932613679365; 0.9563998283455755 1.0 … 0.2173458237443698 0.21693374231808954; … ; 0.21129976679088458 0.2173458237443698 … 1.0 0.9954037096884791; 0.21088932613679365 0.21693374231808954 … 0.9954037096884791 1.0], Bool[0, 1, 1, 0, 1, 1, 1, 1, 1, 0  …  1, 1, 0, 1, 1, 1, 1, 1, 1, 1])

In [44]:
kept_v_b = clump(GenotypesArr[5], chr_pos, 0.01)

994-element Vector{Bool}:
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [45]:
sum(kept_v_b)

7

In [33]:
qtl_data[kept_v_b, :].pval_out

21-element Vector{Union{Missing, Float64}}:
 1.2882495516931322e-32
 1.6595869074375597e-8
 2.570395782768865e-7
 8.317637711026709e-7
 1.479108388168207e-6
 3.0199517204020193e-6
 1.0232929922807536e-5
 2.238721138568338e-5
 3.235936569296281e-5
 3.715352290971728e-5
 4.3651583224016566e-5
 8.317637711026709e-5
 9.332543007969905e-5
 0.0005128613839913648
 0.0005248074602497728
 0.001202264434617413
 0.001584893192461114
 0.0026302679918953813
 0.003090295432513592
 0.0038018939632056127
 0.004073802778041126

In [46]:
qtl_data[kept_v_b, :]

,id_,chr,pos,a_effect_out,a_other_out,β_out,se_out
,identity,identity,identity,identity,identity,identity,identity
,String?,Int8?,Int32?,String?,String?,Float64?,Float64?
1,5_35866218_232320,5,35866218,a,g,-0.2941,0.0247
2,5_35455201_229867,5,35455201,a,g,0.1286,0.0327
3,5_35999069_233108,5,35999069,t,g,-0.5618,0.1438
4,5_35562333_230448,5,35562333,t,c,-0.3956,0.114
5,5_35622908_230828,5,35622908,t,c,-0.1714,0.0512
6,5_36199172_234066,5,36199172,a,g,-0.177,0.056
7,5_35462826_229920,5,35462826,t,g,0.522,0.1769


In [35]:
R2_mat, found = getLDmat(GenotypesArr[5], chr_pos[kept_v_b])

([1.0 0.09282100194408917 … 0.010228097133089581 0.015886371598955014; 0.09282100194408917 1.0 … 0.011172882027492809 0.00018006640010854316; … ; 0.010228097133089581 0.011172882027492809 … 1.0 0.0008856918125756873; 0.015886371598955014 0.00018006640010854316 … 0.0008856918125756873 1.0], Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [36]:
R2_mat

21×21 Matrix{Float64}:
   1.0            0.092821       0.0997207    …    0.0102281      0.0158864
   0.092821       1.0            0.00474854        0.0111729      0.000180066
   0.0997207      0.00474854     1.0               0.000760215    0.019035
   0.0899054      0.0474889      0.0137945         0.0015931      0.00131132
   0.0724797      0.0196626      0.0198181         0.00283715     0.000257418
   0.069828       0.00985312     0.0974703    …    3.15738e-6     0.00750283
   0.0295071      0.0130831      0.0290402         0.00136529     0.00162989
   0.0536659      0.00564676     0.0294596         0.000380563    0.000271872
   0.0807459      0.047652       0.00233034        0.00203686     0.000356443
   0.0694356      0.0777763      0.0102663         0.00436363     0.00128687
   0.0695331      0.0340061      0.0204296    …    0.0030449      2.93375e-5
   0.000416501    0.0174203      0.0629342         0.00349991     0.00103215
   0.00867779     0.0240329      0.00813876        0

In [40]:
v = string.(qtl_data.chr) .*":".* string.(qtl_data.pos) 

994-element Vector{String}:
 "5:35866218"
 "5:35883757"
 "5:35883251"
 "5:35883176"
 "5:35883804"
 "5:35812414"
 "5:35827665"
 "5:35814105"
 "5:35811768"
 "5:35827646"
 "5:35811973"
 "5:35810268"
 "5:35807455"
 ⋮
 "5:35850278"
 "5:35876251"
 "5:35474284"
 "5:35497748"
 "5:35478774"
 "5:35696508"
 "5:35712138"
 "5:35529733"
 "5:35480468"
 "5:35656344"
 "5:35712310"
 "5:35713629"

In [41]:
using DelimitedFiles

writedlm("/mnt/r6_data2/Samuel/ivs.txt", v)

In [43]:
chr_pos[kept_v_b]

21-element Vector{Tuple{Any, Any}}:
 (5, 35866218)
 (5, 35971187)
 (5, 35621200)
 (5, 35945644)
 (5, 35963507)
 (5, 35776239)
 (5, 35761376)
 (5, 35753924)
 (5, 35838799)
 (5, 35833202)
 (5, 35995504)
 (5, 35455201)
 (5, 35999069)
 (5, 35573621)
 (5, 35562333)
 (5, 36051838)
 (5, 36199172)
 (5, 35770132)
 (5, 36090471)
 (5, 36072024)
 (5, 35743409)